In [29]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: tqdm in c:\users\ykhwang\anaconda3\envs\ml\lib\site-packages (4.65.0)



In [30]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

In [53]:
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)

def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        #url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20100131to20211231&start=" + str(start_page)

        
        print('생성url', url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            #url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            
             url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20100131to20211231&start=" + str(page)

            
            urls.append(url)
            print('생성url', url)
        return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url


IndentationError: unexpected indent (2968245614.py, line 29)

In [51]:
#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")   


# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
year_list = []
month_list = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass
    
# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")
    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")
        
    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
        year = str(news_date)[:4]
        month = str(news_date)[5:7]
        year_list.append(int(year))
        month_list.append(int(month))
        print(year, month, news_titles)
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기 2023-07-11 13:13:56
        year = str(news_date)[:4]
        month = str(news_date)[5:7]
        year_list.append(int(year))
        month_list.append(int(month))
        print(year, month, news_titles)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')


###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'year':year_list,'month':month_list})
news_df


#데이터 프레임 저장
news_df.to_csv('{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

검색할 키워드를 입력해주세요:미세먼지

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):21

크롤링할 시작 페이지:  21 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):30

크롤링할 종료 페이지:  30 페이지
생성url https://search.naver.com/search.naver?where=news&sm=tab_pge&query=미세먼지&start=201
생성url https://search.naver.com/search.naver?where=news&sm=tab_pge&query=미세먼지&start=211
생성url https://search.naver.com/search.naver?where=news&sm=tab_pge&query=미세먼지&start=221
생성url https://search.naver.com/search.naver?where=news&sm=tab_pge&query=미세먼지&start=231
생성url https://search.naver.com/search.naver?where=news&sm=tab_pge&query=미세먼지&start=241
생성url https://search.naver.com/search.naver?where=news&sm=tab_pge&query=미세먼지&start=251
생성url https://search.naver.com/search.naver?where=news&sm=tab_pge&query=미세먼지&start=261
생성url https://search.naver.com/search.naver?where=news&sm=tab_pge&query=미세먼지&start=271
생성url https://search.naver.com/search.naver?where=news&sm=tab_pge&query=미세먼지&start=281
생성url https://search.naver.com/search.naver?where=news&sm=tab_pge&query=미세먼지&s

  3%|██▊                                                                                | 1/30 [00:00<00:07,  4.06it/s]

2023 06 ['국내 철강사업장 미세먼지 줄인다']
2023 06 ['국내 철강사업장 미세먼지 줄인다', "'사람의 폐' 닮은 센서로 미세먼지 주범 '이산화질소' 실시간 관측"]


 10%|████████▎                                                                          | 3/30 [00:00<00:06,  4.48it/s]

2023 06 ['국내 철강사업장 미세먼지 줄인다', "'사람의 폐' 닮은 센서로 미세먼지 주범 '이산화질소' 실시간 관측", '정부, 건설현장 미세먼지 줄인다…건설기계 저공해 전환 확대']
2023 06 ['국내 철강사업장 미세먼지 줄인다', "'사람의 폐' 닮은 센서로 미세먼지 주범 '이산화질소' 실시간 관측", '정부, 건설현장 미세먼지 줄인다…건설기계 저공해 전환 확대', '정부, 건설현장 미세먼지 줄인다…건설기계 저공해 전환 확대']


...

2023 06 ['국내 철강사업장 미세먼지 줄인다', "'사람의 폐' 닮은 센서로 미세먼지 주범 '이산화질소' 실시간 관측", '정부, 건설현장 미세먼지 줄인다…건설기계 저공해 전환 확대', '정부, 건설현장 미세먼지 줄인다…건설기계 저공해 전환 확대', '“인체 치명적 지하철 초미세먼지 75% 낮췄다” 철도硏 신기술 개발 성공', '중국발 미세먼지 감시 ‘전초기지’ 40여종 장비가 움직인다', '내일 낮까지 전국 ‘비소식’…미세먼지 ‘좋음’[내일날씨]', '[르포] 중국發 미세먼지 감시 전초기지, 백령도 대기환경연구소를 가다', '우리동네 미세먼지 농도 한눈에…경남 4색 신호등 15대 추가', "충북 한낮 34도 불볕더위…미세먼지 '좋음'", "충북 한낮 34도 불볕더위…미세먼지 '좋음'", "전국 맑고 미세먼지 '좋음'…낮 최고기온 34도", '분진사업장 밀집 동해항 주변 미세먼지 농도·빈도 매우 높아', '“미세먼지 주범 이산화질소” 10억 분의 1까지 잡아낸다', '서울 숨쉬기 편해졌다…초미세먼지 75% 감소, 뭘 바꿨길래', "[오늘날씨] 미세먼지 '좋고' 오후까지 동쪽에 비…낮 최고 32도", '진주 한국남동발전, 미세먼지 저감과 노인일자리 창출을 한번에 해결', '강원대학교, 강원권 미세먼지 연구·관리센터 선정', '[지금은 과학] 국내 철강 산업 미세먼지↓…운영비 30%↓', '[뉴스킹] 홍석준 "日오염수 방류, 우리가 얻는 이익 없어...중국 미세먼지로 국익 있나?"', "충북 맑고 낮 최고 33도 무더위…미세먼지 '보통'", '"미세먼지보다 무서운 오존" 서울 전역에 주의보 발령(종합)', '"미세먼지보다 무서운 오존" 서울 전역에 주의보 발령(종합)', '남원시, 미세먼지 저감 노후경유차 조기폐차 등 지원', "충남권 아침까지 짙은 안개 끼고 낮 30도 이상…미세먼지 '보통'", '"경남 미세먼지 농도, 신호등으로 확인하세요"', '"극초미세먼지 제거" 코웨이, 싱글파워 공기청정기']
2023 06 ['국내 철강사업장 미세먼지 줄인다', "

 97%|███████████████████████████████████████████████████████████████████████████████▎  | 29/30 [00:07<00:00,  3.73it/s]

2023 06 ['국내 철강사업장 미세먼지 줄인다', "'사람의 폐' 닮은 센서로 미세먼지 주범 '이산화질소' 실시간 관측", '정부, 건설현장 미세먼지 줄인다…건설기계 저공해 전환 확대', '정부, 건설현장 미세먼지 줄인다…건설기계 저공해 전환 확대', '“인체 치명적 지하철 초미세먼지 75% 낮췄다” 철도硏 신기술 개발 성공', '중국발 미세먼지 감시 ‘전초기지’ 40여종 장비가 움직인다', '내일 낮까지 전국 ‘비소식’…미세먼지 ‘좋음’[내일날씨]', '[르포] 중국發 미세먼지 감시 전초기지, 백령도 대기환경연구소를 가다', '우리동네 미세먼지 농도 한눈에…경남 4색 신호등 15대 추가', "충북 한낮 34도 불볕더위…미세먼지 '좋음'", "충북 한낮 34도 불볕더위…미세먼지 '좋음'", "전국 맑고 미세먼지 '좋음'…낮 최고기온 34도", '분진사업장 밀집 동해항 주변 미세먼지 농도·빈도 매우 높아', '“미세먼지 주범 이산화질소” 10억 분의 1까지 잡아낸다', '서울 숨쉬기 편해졌다…초미세먼지 75% 감소, 뭘 바꿨길래', "[오늘날씨] 미세먼지 '좋고' 오후까지 동쪽에 비…낮 최고 32도", '진주 한국남동발전, 미세먼지 저감과 노인일자리 창출을 한번에 해결', '강원대학교, 강원권 미세먼지 연구·관리센터 선정', '[지금은 과학] 국내 철강 산업 미세먼지↓…운영비 30%↓', '[뉴스킹] 홍석준 "日오염수 방류, 우리가 얻는 이익 없어...중국 미세먼지로 국익 있나?"', "충북 맑고 낮 최고 33도 무더위…미세먼지 '보통'", '"미세먼지보다 무서운 오존" 서울 전역에 주의보 발령(종합)', '"미세먼지보다 무서운 오존" 서울 전역에 주의보 발령(종합)', '남원시, 미세먼지 저감 노후경유차 조기폐차 등 지원', "충남권 아침까지 짙은 안개 끼고 낮 30도 이상…미세먼지 '보통'", '"경남 미세먼지 농도, 신호등으로 확인하세요"', '"극초미세먼지 제거" 코웨이, 싱글파워 공기청정기', '‘사람의 폐’ 닮은 센서가 미세먼지 주범 ‘이산화질소’

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:08<00:00,  3.64it/s]

2023 06 ['국내 철강사업장 미세먼지 줄인다', "'사람의 폐' 닮은 센서로 미세먼지 주범 '이산화질소' 실시간 관측", '정부, 건설현장 미세먼지 줄인다…건설기계 저공해 전환 확대', '정부, 건설현장 미세먼지 줄인다…건설기계 저공해 전환 확대', '“인체 치명적 지하철 초미세먼지 75% 낮췄다” 철도硏 신기술 개발 성공', '중국발 미세먼지 감시 ‘전초기지’ 40여종 장비가 움직인다', '내일 낮까지 전국 ‘비소식’…미세먼지 ‘좋음’[내일날씨]', '[르포] 중국發 미세먼지 감시 전초기지, 백령도 대기환경연구소를 가다', '우리동네 미세먼지 농도 한눈에…경남 4색 신호등 15대 추가', "충북 한낮 34도 불볕더위…미세먼지 '좋음'", "충북 한낮 34도 불볕더위…미세먼지 '좋음'", "전국 맑고 미세먼지 '좋음'…낮 최고기온 34도", '분진사업장 밀집 동해항 주변 미세먼지 농도·빈도 매우 높아', '“미세먼지 주범 이산화질소” 10억 분의 1까지 잡아낸다', '서울 숨쉬기 편해졌다…초미세먼지 75% 감소, 뭘 바꿨길래', "[오늘날씨] 미세먼지 '좋고' 오후까지 동쪽에 비…낮 최고 32도", '진주 한국남동발전, 미세먼지 저감과 노인일자리 창출을 한번에 해결', '강원대학교, 강원권 미세먼지 연구·관리센터 선정', '[지금은 과학] 국내 철강 산업 미세먼지↓…운영비 30%↓', '[뉴스킹] 홍석준 "日오염수 방류, 우리가 얻는 이익 없어...중국 미세먼지로 국익 있나?"', "충북 맑고 낮 최고 33도 무더위…미세먼지 '보통'", '"미세먼지보다 무서운 오존" 서울 전역에 주의보 발령(종합)', '"미세먼지보다 무서운 오존" 서울 전역에 주의보 발령(종합)', '남원시, 미세먼지 저감 노후경유차 조기폐차 등 지원', "충남권 아침까지 짙은 안개 끼고 낮 30도 이상…미세먼지 '보통'", '"경남 미세먼지 농도, 신호등으로 확인하세요"', '"극초미세먼지 제거" 코웨이, 싱글파워 공기청정기', '‘사람의 폐’ 닮은 센서가 미세먼지 주범 ‘이산화질소’

In [52]:
news_df

,year,month
0,2023,6
1,2023,6
2,2023,6
3,2023,6
4,2023,6
5,2023,6
6,2023,6
7,2023,6
8,2023,6
9,2023,6
